In [0]:
!wget https://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/gzip.zip
!unzip gzip.zip 
!rm gzip.zip
!pip install python-mnist

Source: https://www.nist.gov/node/1298471/emnist-dataset

In [0]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from mnist import MNIST
import os

In [0]:
# load the entire EMNIST dataset as numpy arrays (this might take a while)
emnist_data = MNIST(path='gzip', return_type='numpy')
emnist_data.select_emnist('byclass')
x_train, y_train = emnist_data.load_training()
x_test, y_test = emnist_data.load_testing()

In [0]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [0]:
img_side = 28

# Reshape tensors to [n, y, x, 1] and normalize the pixel values between [0, 1]
x_train = x_train.reshape(-1, img_side, img_side, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, img_side, img_side, 1).astype('float32') / 255.0

x_train.shape, x_test.shape

In [0]:
# get number of classes
unique_classes = np.unique(y_train)
num_classes = len(unique_classes)

input_shape = (img_side, img_side, 1)

# weight the classes (to combat the imbalance)
class_weights = dict(enumerate(compute_class_weight('balanced', unique_classes, y_train)))

# Convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [0]:
kernel_size = (5, 5)
def createmodel():
    return Sequential([
        Convolution2D(16, kernel_size=kernel_size, padding='same', input_shape=input_shape, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Convolution2D(32, kernel_size=kernel_size, padding='same', activation= 'relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Convolution2D(64, kernel_size=kernel_size, padding='same', activation= 'relu'),
        MaxPooling2D(pool_size =(2,2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax'),
    ])

In [0]:
# setting up model to run on cpu, or gpu when avaiable
model = createmodel()
model.compile(loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"])

In [0]:
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

In [0]:
x_train.shape

In [0]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=10,
    restore_best_weights=True)

In [0]:
model.fit(x_train, y_train,
          #class_weight=class_weights,
          batch_size=10000,
          epochs=200,
          verbose=1,
          shuffle=True,
          validation_data=(x_test, y_test),
          callbacks=[es])

In [0]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
import matplotlib.pyplot as plt
def plotres(x, metric):
    plt.plot(model.history.history[metric])
    plt.plot(model.history.history['val_'+metric])
    plt.title(metric.upper())
    plt.ylabel(metric)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

plotres(model.history.history, "acc")
plotres(model.history.history, "loss")

In [0]:
model.save("cnn_emnist.h5")

In [0]:
!pip install tensorflowjs

In [0]:
!rm -rf jsmodel/
!tensorflowjs_converter --input_format keras "cnn_emnist.h5" ./jsmodel
!zip -r jsmodel.zip jsmodel/

In [0]:
y_pred = model.predict(x_test)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import string
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [0]:
labels = string.digits+string.ascii_lowercase+string.ascii_uppercase
plt.subplots(figsize=(20,20))
sns.heatmap(confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)), xticklabels=labels, yticklabels=labels)